# Stephanie's Working Notebook

## Work starting Tuesday, 2/22
Goals for today:
- read Kan's csv file to df to explore with
- add sentiment scores using his algorithm and mine to explore (see questions for explore [here](https://trello.com/c/d7IxpXIS/25-exploratory-questions)
- Come up with minimum x4 Qs and As and Explore Takeaways
- Start adding work we have done to this point to README project info and slideshow shell


<hr>

In [3]:
# python data manipulation and analysis library
import numpy as np

# python library supporting operations on multidimensional arrays and matrices 
import pandas as pd

# python plotting library for visualizations
import matplotlib.pyplot as plt

# data visualization library for python based on matplotlib providing higher level visual interface and options
import seaborn as sns


### Step 1 | Acquire the data

In [4]:
# reading in Kan's prepped csv to df for exploration
df = pd.read_csv('csv_files/sentiment.csv')

In [7]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

df.head()

,file,message,date,content,sender,subject,clean,tokenize,stop_words,stemm,lemmatize,sentiment
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,phillip.allen@enron.com,NaN,here is our forecast\n\n,here is our forecast,forecast,here is our forecast,here is our forecast,0.0000
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,phillip.allen@enron.com,Re:,traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling business meeting takes fun trip espe...,travel to have a busi meet take the fun out of...,traveling to have a business meeting take the ...,0.9313
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,phillip.allen@enron.com,Re: test,test successful way to go,test successful way to go,test successful way go,test success way to go,test successful way to go,0.5859
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s...",phillip.allen@enron.com,NaN,randy\n\n can you send me a schedule of the sa...,randy\n\n can you send me a schedule of the sa...,randy send schedule salary level everyone sche...,randi can you send me a schedul of the salari ...,randy can you send me a schedule of the salary...,0.0000
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,phillip.allen@enron.com,Re: Hello,let's shoot for tuesday at 1145,let ' s shoot for tuesday at 1145,let ' shoot tuesday 1145,let ' s shoot for tuesday at 1145,let ' s shoot for tuesday at 1145,-0.3400


In [10]:
ts = pd.read_csv('csv_files/time_series.csv')

ts.head()

,Unnamed: 0,date,sentiment
0,0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",0.0000
1,1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",0.9313
2,2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",0.5859
3,3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",0.0000
4,4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",-0.3400


##### I want to make sure the rows line up for both dfs before joining them together..

In [11]:
# checking row counts
df.shape[0] == ts.shape[0]

False

In [12]:
df.shape[0]

371388

In [15]:
ts.shape[0]

517401

##### They are not `==`  I will go back and look at why later but, for now, just work with the data read in from the `sentiment.csv` file
### Step 2 | Prepare date for explore
1. drop unnecessary columns
2. handle nulls
3. add on my algorithms sentiment scores

In [16]:
# preview df
df.head(1)

,file,message,date,content,sender,subject,clean,tokenize,stop_words,stemm,lemmatize,sentiment
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,phillip.allen@enron.com,NaN,here is our forecast\n\n,here is our forecast,forecast,here is our forecast,here is our forecast,0.0


#### 2.1 Drop unnecessary columns

Columns to keep
- `date`
- `sender`
- `subject`
- `lemmatize` >> `msg_content`
- `file` >> `file_directory`
- `sentiment`

In [19]:
df = df[['date', 'sender', 'subject', 'lemmatize', 'file', 'sentiment']]

df.head(1)

,date,sender,subject,lemmatize,file,sentiment
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,NaN,here is our forecast,allen-p/_sent_mail/1.,0.0


#### 2.2 Handle nulls

In [26]:
# looking at null counts as a percentage of total data count
(df.isna().sum()/df.shape[0])*100

date         0.000000
sender       0.000000
subject      3.780413
lemmatize    0.008616
file         0.000000
sentiment    0.000269
dtype: float64

**Columns with nulls**
- `subject` >> these are emails without a subject, don't want to drop these, instead will flag by filling `nan` value with str value `no_subject`
- `lemmatize` >> these are emails with no `Nan` or empty email values, small fraction but want to see if anything interesting here >> I want to look at this but will likely flag with str value `empty_text`
- `sentiment` >> no sentiment score, for now such a small % of total will drop these

##### Looking at the rows with `Null` values for the email text

In [28]:
no_text = df[df.lemmatize.isna() == True]

no_text

,date,sender,subject,lemmatize,file,sentiment
40425,"Mon, 11 Dec 2000 03:24:00 -0800 (PST)",don.baughman@enron.com,Bush Country,NaN,campbell-l/all_documents/1208.,0.0
42034,"Mon, 11 Dec 2000 03:24:00 -0800 (PST)",don.baughman@enron.com,Bush Country,NaN,campbell-l/discussion_threads/1084.,0.0
45726,"Mon, 11 Dec 2000 03:24:00 -0800 (PST)",don.baughman@enron.com,Bush Country,NaN,campbell-l/notes_inbox/92.,0.0
52244,"Fri, 26 Oct 2001 11:48:19 -0700 (PDT)",cjewell@bcm.tmc.edu,How to be beautiful all the time,NaN,corman-s/deleted_items/40.,0.0
56591,"Fri, 13 Apr 2001 07:26:00 -0700 (PDT)",marcia.linton@enron.com,TEXT OF BROCHURE ON HI GASPRICES/CALIF. CHAOS,NaN,dasovich-j/all_documents/11093.,0.0
75650,"Fri, 13 Apr 2001 07:26:00 -0700 (PDT)",marcia.linton@enron.com,TEXT OF BROCHURE ON HI GASPRICES/CALIF. CHAOS,NaN,dasovich-j/notes_inbox/4289.,0.0
84398,"Sat, 24 Nov 2001 12:27:57 -0800 (PST)",greg8835@hcqr.yahoo.com,How To Make Money On The Internet! -vgof,NaN,davis-d/deleted_items/173.,0.0
88943,"Tue, 20 Feb 2001 11:54:00 -0800 (PST)",david.delainey@enron.com,Re: Birthday Cards / Welcome Letters for EES E...,NaN,delainey-d/_sent_mail/692.,0.0
89856,"Tue, 20 Feb 2001 11:54:00 -0800 (PST)",david.delainey@enron.com,Re: Birthday Cards / Welcome Letters for EES E...,NaN,delainey-d/all_documents/728.,0.0
91634,"Tue, 20 Feb 2001 11:54:00 -0800 (PST)",david.delainey@enron.com,Re: Birthday Cards / Welcome Letters for EES E...,NaN,delainey-d/sent/697.,0.0


In [30]:
# looking at the sentiment score for these emails
no_text.sentiment.value_counts()

0.0    31
Name: sentiment, dtype: int64

In [31]:
# looking at the sender
no_text.sender.value_counts()

matthew.lenhart@enron.com     9
david.delainey@enron.com      3
don.baughman@enron.com        3
gayla.seiter@enron.com        3
marcia.linton@enron.com       2
dan.hyvl@enron.com            2
legal.7@enron.com             1
greg8835@hcqr.yahoo.com       1
frank.hayden@enron.com        1
cjewell@bcm.tmc.edu           1
gerald.nemec@enron.com        1
louise.kitchen@enron.com      1
tracy.geaccone@enron.com      1
shirley.crenshaw@enron.com    1
rumaldo.lopez@enron.com       1
heather.mathis@enron.com      1
Name: sender, dtype: int64

##### I am okay with dropping these for now. They all have 0 sentiment scores and, unless `matthew.lenhart@enron.com` is a person of interest I do not see where I would need these. 
I am thinking that a lot of the emails may just have attachments?

In [36]:
# filling null subject values with flag >> no_subject
df.subject.fillna('no_subject', inplace = True)

In [37]:
# making sure went through
df[df.subject == 'no_subject']

,date,sender,subject,lemmatize,file,sentiment
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,no_subject,here is our forecast,allen-p/_sent_mail/1.,0.0000
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,no_subject,randy can you send me a schedule of the salary...,allen-p/_sent_mail/1000.,0.0000
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",phillip.allen@enron.com,no_subject,please cc the following distribution list with...,allen-p/_sent_mail/1003.,0.7579
11,"Fri, 13 Oct 2000 06:45:00 -0700 (PDT)",phillip.allen@enron.com,no_subject,lucy here are the rentrolls open them and save...,allen-p/_sent_mail/104.,0.8934
14,"Thu, 5 Oct 2000 06:26:00 -0700 (PDT)",phillip.allen@enron.com,no_subject,dave here are the name of the west desk member...,allen-p/_sent_mail/107.,0.0000
...,...,...,...,...,...,...
370991,"Thu, 24 Jan 2002 12:45:53 -0800 (PST)",emainzer@attbi.com,no_subject,hey richard what is your full mail address in ...,ring-r/deleted_items/282.,0.3400
371068,"Fri, 19 Oct 2001 08:25:22 -0700 (PDT)",richard.ring@enron.com,no_subject,kathy i will be on vacation this afternoon tha...,ring-r/deleted_items/88.,0.4404
371226,"Tue, 16 Oct 2001 12:15:18 -0700 (PDT)",michael.frazier@enron.com,no_subject,here is the bge file i mentioned sorry to be s...,ring-r/eesiisopjm/7.,0.5106
371284,"Wed, 29 Aug 2001 11:36:00 -0700 (PDT)",elliot.mainzer@enron.com,no_subject,how many state are we actively offering direct...,ring-r/eesirenewableenergy/143.,0.5859


In [ ]:
stop_running_notebook_code >> here

## Work done week0

## Acquire
Emails library documentation: [link](https://docs.python.org/3/library/email.html)<br>
Data source: [link](https://www.kaggle.com/wcukierski/enron-email-dataset)

In [1]:
# standard python imports
import numpy as np
import pandas as pd

# imports from python emails library
from email.parser import Parser

In [2]:
# reading csv to df
df = pd.read_csv('../emails.csv')

df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
# getting size of df
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns of data.')

There are 517401 rows and 2 columns of data.


In [4]:
# looking at the first message to begin the process of parsing out the message text
df.message[0]

"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

### The message has some key compents that we can parse out using the `emails` Python Library
- `datetime`
- `sender`
- `recipient`
- `subject`
- `message content` 

In [5]:
# looking to see if pattern matches in other messages
print(df.message[1])
print()
print(df.message[23])
print()
print(df.message[54])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

### Something else to possibly look at for later would be cc and bcc recipients as well as forwarding
If there were any/quantity/who they were/how. many times a message was forwarded

In [6]:
# saving x1 test message to variable to write code for function that will loop through all messages
message = df.message[63]

# function step 1 | getting full message data
message = Parser().parsestr(message)

# looking at output Parsed message object
message

In [7]:
# looking at printout of Parsed message object
print(message)

Message-ID: <1776521.1075855688576.JavaMail.evans@thyme>
Date: Wed, 6 Sep 2000 04:46:00 -0700 (PDT)
From: phillip.allen@enron.com
To: thomas.martin@enron.com, mike.grigsby@enron.com, keith.holst@enron.com,
	jay.reitmeyer@enron.com, frank.ermis@enron.com
Subject: Wow
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Thomas A Martin, Mike Grigsby, Keith Holst, Jay Reitmeyer, Frank Ermis
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 
10:49 AM ---------------------------


Jeff Richter
09/06/2000 07:39 AM
To: Phillip K Allen/HOU/ECT@ECT
cc:  
Subject: Wow


---------------------- Forwarded by Jeff Richter/HOU/ECT on 09/06/2000 09:45 
AM ---------------------------
To: Mike Swerzbin/HOU/ECT@ECT, Robert Badeer/HOU/ECT@ECT, Sean 
Crandall/PDX/ECT@ECT, Tim Belden/HOU/ECT@ECT, Jeff 

In [8]:
# looking at the message dtype
type(message)

email.message.Message

### We can use the [email.message.Message](https://docs.python.org/3/library/email.compat32-message.html) documents to further parse the message info

In [9]:
# parsed message text content
text = message.get_payload()
text

'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 \n10:49 AM ---------------------------\n\n\nJeff Richter\n09/06/2000 07:39 AM\nTo: Phillip K Allen/HOU/ECT@ECT\ncc:  \nSubject: Wow\n\n\n---------------------- Forwarded by Jeff Richter/HOU/ECT on 09/06/2000 09:45 \nAM ---------------------------\nTo: Mike Swerzbin/HOU/ECT@ECT, Robert Badeer/HOU/ECT@ECT, Sean \nCrandall/PDX/ECT@ECT, Tim Belden/HOU/ECT@ECT, Jeff Richter/HOU/ECT@ECT, John \nM Forney/HOU/ECT@ECT, Matt Motley/PDX/ECT@ECT, Tom Alonso/PDX/ECT@ECT, Mark \nFischer/PDX/ECT@ECT\ncc:  \nSubject: Wow\n\n\n---------------------- Forwarded by Tim Belden/HOU/ECT on 09/06/2000 07:27 AM \n---------------------------\n   \n\tEnron Capital & Trade Resources Corp.\n\t\n\tFrom:  Kevin M Presto                           09/05/2000 01:59 PM\n\t\n\nTo: Tim Belden/HOU/ECT@ECT\ncc: Rogers Herndon/HOU/ECT@ect, John Zufferli/HOU/ECT@ECT, Lloyd \nWill/HOU/ECT@ECT, Doug Gilbert-Smith/Corp/Enron@ENRON, Mike \nSwerzbin/HOU/ECT

In [10]:
# looking at text printout
print(text)

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 
10:49 AM ---------------------------


Jeff Richter
09/06/2000 07:39 AM
To: Phillip K Allen/HOU/ECT@ECT
cc:  
Subject: Wow


---------------------- Forwarded by Jeff Richter/HOU/ECT on 09/06/2000 09:45 
AM ---------------------------
To: Mike Swerzbin/HOU/ECT@ECT, Robert Badeer/HOU/ECT@ECT, Sean 
Crandall/PDX/ECT@ECT, Tim Belden/HOU/ECT@ECT, Jeff Richter/HOU/ECT@ECT, John 
M Forney/HOU/ECT@ECT, Matt Motley/PDX/ECT@ECT, Tom Alonso/PDX/ECT@ECT, Mark 
Fischer/PDX/ECT@ECT
cc:  
Subject: Wow


---------------------- Forwarded by Tim Belden/HOU/ECT on 09/06/2000 07:27 AM 
---------------------------
   
	Enron Capital & Trade Resources Corp.
	
	From:  Kevin M Presto                           09/05/2000 01:59 PM
	

To: Tim Belden/HOU/ECT@ECT
cc: Rogers Herndon/HOU/ECT@ect, John Zufferli/HOU/ECT@ECT, Lloyd 
Will/HOU/ECT@ECT, Doug Gilbert-Smith/Corp/Enron@ENRON, Mike 
Swerzbin/HOU/ECT@ECT 
Subject: Wow

Do not underestim

In [11]:
# parsing sender
sender = message['From']
sender

'phillip.allen@enron.com'

In [12]:
# parsing recipient
recip = message['To']
recip

'thomas.martin@enron.com, mike.grigsby@enron.com, keith.holst@enron.com, \n\tjay.reitmeyer@enron.com, frank.ermis@enron.com'

### `recip` will be something to possibly look at post MVP
- number of recipients
- frequency of certainr recipient groups

In [13]:
# parsing message date
date = message['Date']
date

'Wed, 6 Sep 2000 04:46:00 -0700 (PDT)'

In [14]:
# # running dataframe message contents through for loop that will above code to parse contents and save to lists

# # empty lists to be appended in for loop and added to df later
# content = []
# date = []
# sender = []

# # for loop 
# for string in df.message:
#     message = Parser().parsestr(string)
#     msg_content = message.get_payload()
#     msg_date = message['Date']
#     msg_sender = message['From']
#     # adding to empty lists
#     content.append(msg_content)
#     date.append(msg_date)
#     sender.append(msg_sender)

### Skipping ahead and using code from `acquire.pj.py` file in PJ's folder

In [15]:
def acquire_emails():
    df = pd.read_csv('../emails.csv')

    bodies = []
    dates = []

    # loop through email messages
    for i in df.message:
        # parse and set message to email data type
        headers = Parser().parsestr(i)
        # get the body text of the email
        body = headers.get_payload()
        # get the date from email
        date = headers['Date']
        # append date and body text to lists
        bodies.append(body)
        dates.append(date)

    # Set lists to dataframes
    body_df = pd.DataFrame(bodies, columns = ['Content'])
    dates_df = pd.DataFrame(dates, columns = ['Content'])

    # Insert those data frames into our orignal dataframe
    df.insert(1, "content", body_df)
    df.insert(1, "date", dates_df)

    return df

In [16]:
raw_data = pd.read_csv('raw_data.csv')

raw_data.head()

,Unnamed: 0,file,date,content,message
0,0,allen-p/_sent_mail/1.,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,Message-ID: <18782981.1075855378110.JavaMail.e...
1,1,allen-p/_sent_mail/10.,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,Message-ID: <15464986.1075855378456.JavaMail.e...
2,2,allen-p/_sent_mail/100.,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,Message-ID: <24216240.1075855687451.JavaMail.e...
3,3,allen-p/_sent_mail/1000.,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s...",Message-ID: <13505866.1075863688222.JavaMail.e...
4,4,allen-p/_sent_mail/1001.,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,Message-ID: <30922949.1075863688243.JavaMail.e...


### For prepare using `stem` for MVP because it is faster. 
Also saving removing stop words for post MVP work
- lowercase
- dtypes
- nulls

In [17]:
import prepare_steph as prepare

In [18]:
def basic_clean(string):
    '''
    This function takes in a string and
    returns the string normalized.
    '''
    string = unicodedata.normalize('NFKD', string)\
            .encode('ascii', 'ignore')\
            .decode('utf-8', 'ignore')
    string = re.sub(r'[^\w\s]', '', string).lower()
    return string

In [19]:
raw_data.content[54]

'why is aeco basis so low on the list?  Is NWPL mapped differently than AECO?  \nWhat about the correlation to Nymex on AECO?'

In [20]:
import unicodedata
import re
import json
import os

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import acquire_pj
from time import strftime

from sklearn.model_selection import train_test_split

basic_clean(raw_data.content[54])

'why is aeco basis so low on the list  is nwpl mapped differently than aeco  \nwhat about the correlation to nymex on aeco'

In [21]:
print(basic_clean(raw_data.content[54]))

why is aeco basis so low on the list  is nwpl mapped differently than aeco  
what about the correlation to nymex on aeco


In [22]:
raw_data.head(10)['content'].apply(basic_clean)

0                            here is our forecast\n\n 
1    traveling to have a business meeting takes the...
2                           test successful  way to go
3    randy\n\n can you send me a schedule of the sa...
4                     lets shoot for tuesday at 1145  
5    greg\n\n how about either next tuesday or thur...
6    please cc the following distribution list with...
7                      any morning between 10 and 1130
8    1 login  pallen pw ke9davis\n\n i dont think t...
9     forwarded by phillip k allenhouect on 1016200...
Name: content, dtype: object

In [23]:
# # caching raw data
# raw_data.to_csv('raw_data.csv')

In [4]:
import pandas as pd 

df_2 = pd.read_csv('clean_data.csv')

In [7]:
df_2.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], inplace = True)

In [8]:
df_2.head()

,file,date,content,message,clean
0,allen-p/_sent_mail/1.,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,Message-ID: <18782981.1075855378110.JavaMail.e...,here is our forecast
1,allen-p/_sent_mail/10.,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,Message-ID: <15464986.1075855378456.JavaMail.e...,travel to have a busi meet take the fun out of...
2,allen-p/_sent_mail/100.,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,Message-ID: <24216240.1075855687451.JavaMail.e...,test success way to go
3,allen-p/_sent_mail/1000.,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s...",Message-ID: <13505866.1075863688222.JavaMail.e...,randi can you send me a schedul of the salari ...
4,allen-p/_sent_mail/1001.,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,Message-ID: <30922949.1075863688243.JavaMail.e...,let shoot for tuesday at 1145


### Sentiment Analysis Code

In [9]:
# imports
import math
import re
import string
from itertools import product

import nltk.data
from nltk.util import pairwise

In [39]:
df_2.clean.dropna(inplace = True)

## terminal command to install textblob library >> `pip install textblob`
Textblob sentiment analyzer returns two properties for a given input sentence: 
- `Polarity` is a float that lies between [-1,1], 
    - -1 indicates negative sentiment and +1 indicates positive sentiments. 
- `Subjectivity` is also a float which lies in the range of [0,1]. 
    - Subjective sentences generally refer to personal opinion, emotion, or judgment. 
    
[Info Source](https://neptune.ai/blog/sentiment-analysis-python-textblob-vs-vader-vs-flair#:~:text=Textblob%20sentiment%20analyzer%20returns%20two,of%20%5B0%2C1%5D.)

In [42]:
# installing textblob library for sentiment analysis scoring
from textblob import TextBlob

In [103]:
# writing code for one content to test out scoring function
test_1 = df_2.clean[1]
test_2 = df_2.clean[4]
test_3 = df_2.clean[88]

In [104]:
print(TextBlob(test_1).sentiment)
print('-----------------------')
print(test_1)

Sentiment(polarity=0.17045454545454544, subjectivity=0.4007575757575758)
-----------------------
travel to have a busi meet take the fun out of the trip especi if you have to prepar a present i would suggest hold the busi plan meet here then take a trip without ani formal busi meet i would even tri and get some honest opinion on whether a trip is even desir or necessari as far as the busi meet i think it would be more product to tri and stimul discuss across the differ group about what is work and what is not too often the present speak and the other are quiet just wait for their turn the meet might be better if held in a round tabl discuss format my suggest for where to go is austin play golf and rent a ski boat and jet ski fli somewher take too much time


In [67]:
sent, sub = TextBlob(test_3).sentiment

In [68]:
sent

0.055026455026455014

In [74]:
def sentiment_scores(string):
    '''
    
    '''
    
    polarity, subjectivity = TextBlob(string).sentiment
    
    return polarity, subjectivity

In [110]:
def add_scores(df, clean_msg_col):
    '''
    
    '''
    
    df['polarity, subjectivity'] = df[clean_msg_col].apply(sentiment_scores)
    
    pol = []
    subj = []
    for tuple_ in df['polarity, subjectivity']:
        pol.append(list(tuple_)[0])
        subj.append(list(tuple_)[1])
    
#     df = df.drop(columns = ['polarity, sentiment'])
    df['polarity'] = pol
    df['subjectivity'] = sent
    
    return df

In [112]:
test_df = add_scores(df_2.head(50), 'clean')
test_df[['clean', 'polarity, subjectivity']]

<ipython-input-110-0b8db8501343>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity, subjectivity'] = df[clean_msg_col].apply(sentiment_scores)
<ipython-input-110-0b8db8501343>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = pol
<ipython-input-110-0b8db8501343>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,clean,"polarity, subjectivity"
0,here is our forecast,"(0.0, 0.0)"
1,travel to have a busi meet take the fun out of...,"(0.17045454545454544, 0.4007575757575758)"
2,test success way to go,"(0.3, 0.0)"
3,randi can you send me a schedul of the salari ...,"(0.0, 0.0)"
4,let shoot for tuesday at 1145,"(0.0, 0.0)"
5,greg how about either next tuesday or thursday...,"(0.0, 0.0)"
6,pleas cc the follow distribut list with updat ...,"(0.0, 0.0)"
7,ani morn between 10 and 1130,"(0.0, 0.0)"
8,1 login pallen pw ke9davi i dont think these a...,"(0.5, 0.9)"
9,forward by phillip k allenhouect on 10162000 0...,"(0.0058333333333333345, 0.30333333333333334)"


In [101]:
test_df.clean[1]

'travel to have a busi meet take the fun out of the trip especi if you have to prepar a present i would suggest hold the busi plan meet here then take a trip without ani formal busi meet i would even tri and get some honest opinion on whether a trip is even desir or necessari as far as the busi meet i think it would be more product to tri and stimul discuss across the differ group about what is work and what is not too often the present speak and the other are quiet just wait for their turn the meet might be better if held in a round tabl discuss format my suggest for where to go is austin play golf and rent a ski boat and jet ski fli somewher take too much time'

In [115]:
test_df[test_df.polarity == test_df.polarity.max()]

,file,date,content,message,clean,"polarity, subjectivity",polarity,subjectivity
21,allen-p/_sent_mail/113.,"Tue, 3 Oct 2000 03:17:00 -0700 (PDT)",I think Fletch has a good CPA. I am still doi...,Message-ID: <17610321.1075855687735.JavaMail.e...,i think fletch ha a good cpa i am still do my own,"(0.6499999999999999, 0.8)",0.65,0.055026


In [116]:
test_df.content[21]

'I think Fletch has a good CPA.  I am still doing my own.  '

In [117]:
test_df[test_df.polarity == test_df.polarity.min()]

,file,date,content,message,clean,"polarity, subjectivity",polarity,subjectivity
10,allen-p/_sent_mail/103.,"Mon, 16 Oct 2000 06:42:00 -0700 (PDT)","Mr. Buckner,\n\n For delivered gas behind San ...",Message-ID: <33076797.1075855687515.JavaMail.e...,mr buckner for deliv ga behind san diego enron...,"(-0.4, 0.7)",-0.4,0.055026


In [121]:
test_df.content[27]

'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \n02:00 PM ---------------------------\n\n\n\tReschedule\nChairperson: Richard Burchfield\nSent by: Cindy Cicchetti\n\nStart: 10/03/2000 02:30 PM\nEnd: 10/03/2000 03:30 PM\n\nDescription: Gas Physical/Financail Positions - Room 2537\n\n\n\nThis meeting repeats   starting on    (if the date occurs on a weekend the \nmeeting ).\nMeeting Dates: \n\n\n\nFletcher J Sturm/HOU/ECT\nScott Neal/HOU/ECT\nHunter S Shively/HOU/ECT\nPhillip K Allen/HOU/ECT\nAllan Severude/HOU/ECT\nScott Mills/HOU/ECT\nRuss Severson/HOU/ECT\n\nDetailed description:\n\n\n\n---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \n02:00 PM ---------------------------\n\n\n\tConfirmation\nChairperson: Richard Burchfield\nSent by: Cindy Cicchetti\n\nStart: 10/03/2000 02:30 PM\nEnd: 10/03/2000 03:30 PM\n\nDescription: Gas Physical/Financail Positions - Room 2537\n\n\n\nThis meeting repeats   starting on    (if the date occurs on 